In [ ]:
import numpy as np

import matplotlib.pyplot as plt

np.random.seed(42)

## Önsöz

Merhaba arkadaşlar, bu noktaya kadar "Least Squares Method"u 0'dan türettik fakat yaptığımız her şey kağıt üstünde formülizasyondan ibaretti.
Bu noktaya kadarki kısım çok kıymetli olsa da en nihayetinde "Least Squares Method" bilgisayarda çalışması gereken bir algoritma. O yüzden bu ufak
egzersizde "Least Squares Method"un kapalı formda çözümünü koda dökeceğiz.

**Anahtar Kelimeler:** *Least Squares Method, Linear Regression, Closed Form Solution*

### İşleyiş

Bu "notebook"da bazı kısımları sizlerin doldurması için boş bıraktım. Sadece bu kısımları doldurun, tüm "cell"leri yukarıdan aşağı çalıştırın ve eğlenmenize bakın :)

Basit bir örnek ile başlayalım, README.md dosyasındaki örneği hatırlayın, 2 boyutlu koordinat sisteminde noktalar var ve biz bu noktalara uygun
fonksiyonu bulmaya çalışıyoruz. Gelin aynı örneği kodlayalım.

In [ ]:
# Data Creation

# Bu kısımda bize uygun bir veri seti oluşturuyoruz. Burada sizin müdahele etmenizi gerektiren bir kısım yok fakat yine de anlamak için
# inceleyebilirsiniz.

x_range = (-10, 10) # x değerlerinin alabileceği aralık
n = 100 # veri setindeki örnek sayısı

# Şimdi noktalarımızın takip etmesini istediğimiz 2 boyutlu lineer fonksiyonu tanımlayalım
m = 2 # eğim
b = 5 # y eksenini kestiği nokta

# Şimdi x için tanımladığımız aralıkta n adet rasgele sayı üretelim
x = np.random.uniform(x_range[0], x_range[1], n)

# Şimdi y değerlerini hesaplayalım
y = m * x + b # Bildiğiniz doğru denklemi

# Bu noktada (x, y) şeklinde noktaları plot edersek kusursuz bir doğru elde ederiz
plt.plot(x, y, 'o')
plt.title("Gürültü eklenmemiş veri seti")
plt.show()

# Fakat bu veri seti oldukça basit ve gerçek hayattaki veri setlerinin aksine kusursuz bir doğru elde ettik.
# O zaman bu veri setine biraz gürültü ekleyelim. Bunun için y değerlerine "Gaussian Distribution" kullanarak gürültü ekleyeceğiz. (Bknz: https://en.wikipedia.org/wiki/Normal_distribution)

y_noise = np.random.normal(0, 5, n) # ortalama 0, standart sapma 5 olan Gaussian Distribution'dan n adet rasgele sayı üretiyoruz
y = y + y_noise # y değerlerine gürültüyü ekliyoruz

# Şimdi veri setimizi tekrar plot edelim
plt.plot(x, y, 'o')
plt.title("Gürültü eklenmiş veri seti")
plt.show()


In [ ]:
# Bu kısım tamamen ne olduğu bildiğimiz bir fonksiyonu nasıl çizdiririz bununla alakalı, least squares ile alakalı değil. Genel bir bilgi.

# Genelde herhangi bir fonksiyonu plot etmek istediğimizde belli bir aralıkta yüksek miktarda eşit aralıklı nokta üretiriz.
# Aşama aşama mantığı göstermemiz gerekirse nokta sayısı n = 2, 5, 10, 20, 50, 100 ve 500 için plot edelim.

# Öncelikle x değerlerini üretelim
x_plot = [ np.linspace(x_range[0], x_range[1], n_plot) for n_plot in [2, 5, 10, 20, 50, 100, 500] ]

# Şimdi y değerlerini hesaplayalım
y_plot = [ m * x + b for x in x_plot ]

# Şimdi plot edelim
for i in range(len(x_plot)):
    plt.subplot(3, 3, i + 1)
    plt.scatter(x_plot[i], y_plot[i], s=2)
    plt.title("n = " + str(len(x_plot[i])))
plt.show()


Görüldüğü gibi çizdirmek istediğimiz plot üstünde ne kadar fazla noktayı çizersek o kadar sanki devamlı bir fonksiyon gibi gözüken plot elde ederiz. Bu genel bir tekniktir, "notebook"un kalanında da bolca kullanacağımız için göstermiş olalım.

In [ ]:
# Datayı kendi seçtiğimiz ama ilerleyen kısımda bilmediğimizi varsaydığımız bir doğruya göre üretmiştik, şimdi o doğruyu çizelim.
# Çizdiğimiz bu doğru "least squares method" ile bulmaya çalışacağımız doğru olacak.

plt.scatter(x, y)
x_plot = np.linspace(x_range[0], x_range[1], 1000)
y_plot = m * x_plot + b

plt.scatter(x_plot, y_plot, color='red', s=0.5)

plt.title('Datayı ifade eden gerçek doğru ve ürettiğimiz data')

plt.show()

Buraya kadarki kısım bir direkt olarak "Least Squares Method"un uygulanması ile alakalı değildi, sadece işin görselleştirme ve datasetleri tanımlama kısmıydı, şimdi asıl kısıma geçelim.

### Least Squares Method

Problemimizi kısaca özetleyelim, herhangi bir $\textbf{w} \in \mathbb{R}^D$ parametre vektörü için, $\textbf{X}$ matrisindeki her bir elemandan elde ettiğimiz tahminleri $\hat{\textbf{Y}} = \textbf{X}\textbf{w}$ olarak göstermiştik.
Denediğimiz herhangi bir $\textbf{w}$ için, tahminlerin gerçeğe ne kadar yakın olduğunu görmek adına da bir de hata fonksiyonu tanımlamıştık, bu fonksiyonu $\textbf{w}$'nin bir fonksiyonu olarak yazarsak:

$E: \mathbb{R}^D \rightarrow \mathbb{R}, \quad E(\textbf{w}) = \sum_{i=1}^N (\textbf{Y}_i - \hat{\textbf{Y}}_i)^2 = (\textbf{Y} - \hat{\textbf{Y}})^T(\textbf{Y} - \hat{\textbf{Y}}) = (\textbf{Y} - \textbf{X}\textbf{w})^T(\textbf{Y} - \textbf{X}\textbf{w})$

In [ ]:
# Gelin bu hata fonksiyonunu bir python fonksiyonu olarak tanımlayalım.

def E(w: np.ndarray, X: np.ndarray, Y: np.ndarray) -> float:
    """ Bu fonksiyon parametre olarak aldığı w ağırlık vektörü için
    ortalama hata kareler toplamını (mean squared error) hesaplayıp geri döndürsün.

    Önemli not 1: For veya while döngüsü kullanmadan, sadece matris işlemleri ile bu fonksiyonu yazın.

    Args:
        w (numpy.ndarray): Ağırlık vektörü. Boyutu D x 1.

    Returns:
        float: Hata değeri.
    """

    err = 0.0 # Hesaplandıktan sonra döndürülecek hata değeri

    # TODO: Hata değerini hesaplayın ve err değişkenine atayın.

    #################################################
    ########## KODUNUZU BURAYA YAZINIZ ##############
    #################################################

    return err

In [ ]:
print("1. Test")
print("Hatayı doğru hesaplayabiliyor muyuz?")

test_X = np.array([[ 0.04091918, -1.00218746], [ 0.74082435, -0.51321357], [-0.22859992, -0.99434937]])
test_Y = np.array([[-2.56233366], [-0.19102776], [ 2.41261542]])
test_ws = [ np.array([[1.0, 2.0]]).T, np.array([[3.0, 4.0]]).T, np.array([[5.0, 6.0]]).T, np.array([[7.0, 8.0]]).T, np.array([[9.0, 10.0]]).T ]

expected_errors = [ 21.80370359, 51.94926291, 101.86640895, 171.55514171, 261.01546118 ]

for i in range(len(test_ws)):
    print("Test", i+1, ":", end=" ")
    error = E(test_ws[i], test_X, test_Y)
    if np.isclose(error, expected_errors[i]):
        print("BAŞARILI")
    else:
        print("ÇUVALLADI: Beklenen", expected_errors[i], "elde edilen", error)


### Eklememiz Gereken Ufak Bir Detay:

Şimdiye kadar herhangi bir $x_i \in \mathbb{R}^{D \times 1}$ verisi için karşılık gelen $\hat{y}_i$ değerini tahmin etmek için $\hat{y}_i = x_i^Tw$ ifadesini kullandık. (X ve Y matrislerini nasıl tanımladığımızı düşününün, X'in tek bir satırı için gösterim bu olur.)

Bu gösterim aslında birçok açıdan doğru, ama bazı özel durumlar için biraz daha açıklama gerektiriyor. Mesela bizim doğru bulma örneğimizi düşünelim.
Bulmak istediğimiz doğru denklemi $y = m \times x + b$ formatında (bildiğimiz 2 boyutlu doğru) yani sadece 1 adet $x$ değeri için karşılık gelen bir $y$ değeri var.
O zaman bunları matris formuna getirecek olursak $D=1$ olarak modellememiz gerekir ve $\textbf{X} \in \mathbb{R}^{N \times 1}, \textbf{Y} \in \mathbb{N}, \textbf{w} \in \mathbb{R}^{1 \times 1}$ olur.
Fakat burada çok büyük bir problem var, $\textbf{w}$ bir skaler! Yani ortada bilinmeyen sadece tek bir parametre var, ama $y = m \times x + b$ denkleminde bulmamız gereken 2 parametre (m ve b) var!
Demek ki bir şeyleri yanlış yapıyoruz.

Aslında tüm problem konunun en başında yaptığımız bir __basitleştirmeden__ kaynaklı, çok boyutlu senaryoya dönersek şunu demiştik:

Tek bir $\textbf{x} \in \mathbb{R}^{D}$ vektörü ve parametre vektörü $\textbf{w} \in \mathbb{R}^{D}$ için $\hat{y} = \textbf{x}^T\textbf{w} = \sum_{i=1}^D \textbf{x}_i \times \textbf{w}_i$ ifadesini kullanarak $\hat{y}$ değerini tahmin edebiliriz.

Bu ifadede __tüm terimler__ $x$ __lere bağlı__! Ama $y = m \times x + b$ denkleminde $b$, $x$'e bağlı değil, yani bizim modelimiz bu denklemi ifade edemiyor!!!

İşte burada kendi problemimizdeki $x$ leri farklı bir şey olarak görmeye ihtiyacımız var, başta demiştik ki $\textbf{x}$ vektörü herhangi obje tipi ile ilgili sayısal veriler barındırır, mesela arsa alanı, ağaç sayısı, en, boy gibi demiştik. Bu durumda ne kadar ölçümümüz varsa $\textbf{x}$ o kadar boyutlu oluyordu. Yani bizim ölçtüğümüz özellik sayısı $D$ ye eşitti, ama bu bizi belli şeyleri yapmaktan alıkoyuyor.

O zaman şöyle düşünsek nasıl olur $\textbf{x}$ sadece yaptığımız ölçümler olmasın, içine alaksız şeyler de koyabilelim, eklediğimiz her yeni şey için de $D$'yi ona göre arttıralım ve bu her yeni şeye de karşılık gelen bir $\textbf{w}_i$ değeri olsun. Örneğin bir arsayı ifade eden $\textbf{x}$ vektörüne yeni bir eleman ekleyelim $1$ bildiğiniz dümdüz $1$. Yani yeni $\textbf{x}$ :

$\textbf{x}_{yeni} = [ \textbf{x}_1 ,\textbf{x}_2, ..., \textbf{x}_D, 1 ]^T $ şeklinde bir vektör olacak, bu bize ne kazandırır? Artık tahmin edilen $\hat{y}$ :

$\hat{y} = \textbf{x}_{yeni}^T\textbf{w}_{yeni} = \textbf{x}_1 \times \textbf{w}_1 + \textbf{x}_2 \times \textbf{w}_2 + ... + \textbf{x}_D \times \textbf{w}_{D} + 1 \times \textbf{w}_{D+1}$

$= \textbf{x}_1 \times \textbf{w}_1 + \textbf{x}_2 \times \textbf{w}_2 + ... + \textbf{x}_D \times \textbf{w}_{D} + \textbf{w}_{yeni \space D+1}$

Yeni eklenen $\textbf{w}_{yeni \space D+1}$ parametresi orijinal $\textbf{x}$ e bağlı değil!!! Yani yeni modelimiz artık $y = m \times x + b$ denklemini de ifade edebilir!

Bu yeni tekniği daha da genelleştirebiliriz, örneğin $x \in \mathbb{R}$ sayısının $P$ cinsinden bir polinomuna bağlı bir y öğrenmek istiyoruz, yani $y = a_0 + a_1 \times x + a_2 \times x^2 + ... + a_P \times x^P$ denklemi için $\textbf{x} \in \mathbb{R}^{P+1}$ ve $\textbf{w} \in \mathbb{R}^{P+1}$ olur. Bu durumda $\textbf{x}$ vektörümüzü şöyle tanımlayabiliriz:

$\textbf{x} = [1, x, x^2, ..., x^P]^T$

Buna karşılık gelen $\textbf{w}$ vektörü de:

$\textbf{w} = [a_0, a_1, a_2, ..., a_P]^T$

Yani anlayacağınız $\textbf{x}$ e lineer olarak bağlı olmayan fonksiyonları bile öğrenmemiz aslında mümkün, tek gereken ihtiyacımız olan her türlü terimi $\textbf{x}$ vektörüne eklemek ve ona karşılık gelen $\textbf{w}$ parametrelerini öğrenmek.

### Bizim Probleme Dönersek

$y = m \times x + b$ formunda bir fonksiyonu öğrenmemiz için $\textbf{x} = [x, 1]^T$ ve $\textbf{w} = [m, b]^T$ olur. Yani $\textbf{x}$ vektörümüzü ölçümlerimizden oluşan bir vektör ve $1$'den oluşan bir vektörün birleşimi olarak düşünebiliriz. $\textbf{w}$ vektörümüz ise $m$ ve $b$ parametrelerimizi içeriyor. Artık problemi çözmek için gereken her şey elimizde!

In [ ]:
# Bu kısımda başta yarattığımız x ve y değerlerini istediğimiz formata dönüştürelim.

# Hatırlayın sadece tek bir ölçümümüz var, noktamızın x eksenindeki konumu, buna göre y'yi bulmak istiyoruz, bunun için x'e bağlı olmayan ikinci bir parametre öğrenmek istiyoruz.
x = x.reshape(-1, 1) # Öncelikle x değerlerini N x 1 boyutlu bir matrise dönüştürüyoruz
ones = np.ones((n, 1)) # N x 1 boyutlu 1'lerden oluşan bir matris oluşturuyoruz

# Şimdi eğer bu iki matrisi yan yana birleştirirsek N x 2 boyutlu bir matris elde ederiz
# Bu matrisin her bir satırında x değerimiz ve 1 değeri olacak yani önceden tartıştığımız formüldeki x ve 1 değerleri
X = np.concatenate((x, ones), axis=1)

# Şimdi de y değerlerini N x 1 boyutlu bir matrise dönüştürüyoruz
Y = y.reshape(-1, 1)

In [ ]:
print("2. Test")
print("Matrisleri doğru yarattık mı ve hata fonksiyonu bu örneğimizde de doğru çalışıyor mu?")

print()
print("Yarattığımız yeni X'in ilk 5 satırı:")
print(X[:5, :])

print()
print("Yarattığımız yeni Y'in ilk 5 satırı:")
print(Y[:5, :])

test_ws = [ np.array([[1.0, 2.0]]).T, np.array([[3.0, 4.0]]).T, np.array([[5.0, 6.0]]).T, np.array([[7.0, 8.0]]).T, np.array([[9.0, 10.0]]).T ]

expected_errors = [ 5336.84446228, 6626.14500242, 36082.55356589, 93706.07015268, 179496.69476279 ]

print()

for i in range(len(test_ws)):
    print("Test", i+1, ":", end=" ")
    error = E(test_ws[i], X, Y)
    if np.isclose(error, expected_errors[i]):
        print("BAŞARILI")
    else:
        print("ÇUVALLADI: Beklenen", expected_errors[i], "elde edilen", error)

In [ ]:
# X ve Y matrislerini alıp en iyi w'yu döndüren fonksiyou yazalım.

def find_best_w(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    """ Bu fonksiyon parametre olarak aldığı X ve Y matrislerini kullanarak
    en iyi w değerini bulup geri döndürür.

    Args:
        X (numpy.ndarray): X matrisi. Boyutu N x D.
        Y (numpy.ndarray): Y matrisi. Boyutu N x 1.

    Returns:
        numpy.ndarray: En iyi w değeri. Boyutu D x 1.
    """

    w = np.zeros((X.shape[1], 1)) # En başta w değerimizi 0'lar ile başlatalım.

    # TODO: w değerini bulun ve w değişkenine atayın.

    #################################################
    ########## KODUNUZU BURAYA YAZINIZ ##############
    #################################################

    return w

In [ ]:
print("3. Test")
print("Optimal w'yu bulabiliyor muyuz?")

w = find_best_w(X, Y) # En iyi w değerini bulalım
err = E(w, X, Y) # Bulduğumuz w değeri ile hatayı hesaplayalım

print()
print("Bulduğumuz w değeri:")
print(w)

print()
print("Gerçek w değeri:")
print(np.array([[2.0], [5.0]])) # m = 2, b = 5 yani gerçek w değerimiz bu

print()
print("Bulduğumuz w değeri ile elde ettiğimiz hata:")
print(err)

print()
print("Gerçek w değeri ile elde etmemiz gereken hata:")
print(2016.46140992)

print()
print("Öğrendiğimiz fonksiyonun grafiği:")

x_plot = np.linspace(-10, 10, 1000) # -10 ile 10 arasında 1000 tane x değeri oluşturalım
y_plot_pred = w[0] * x_plot + w[1] # Bu x değerlerine karşılık gelen y değerlerini hesaplayalım
y_plot_real = 2 * x_plot + 5 # Gerçek y değerlerini hesaplayalım

plt.scatter(X[:, 0], Y[:, 0], label="Veri", s=3) # Verileri çizelim
plt.scatter(x_plot, y_plot_pred, label="Öğrenilen fonksiyon", s=0.5) # Öğrenilen fonksiyonun grafiğini çizelim
plt.scatter(x_plot, y_plot_real, color="red", label="Gerçek fonksiyon", s=0.5) # Gerçek fonksiyonun grafiğini çizelim
plt.legend() # Grafiğin sağ üstünde açıklamaları gösterelim
plt.show() # Grafiği gösterelim




Gördüğünüz gibi $\textbf{w}$ yu kusursuz bir şekilde tahmin edemedik, ama bu çok normal. En nihayetinde veriyi oluştururken biraz gürültü eklemiştik, bu gürültüyü de modelimiz öğrenmiş oldu. Ama yine de modelimiz veriyi oldukça iyi öğrenmiş, yani veriyi oluşturan gerçek fonksiyonu oldukça iyi yakalamış. Bu da kabul edilmesi gereken önemli bir nokta, herhangi bir machine learning modeli verisi ne kadar iyiyse en fazla o kadar iyi olabilir ama daha iyi olamaz. Çok gürültülü bir veri seti ile en iyi machine learning modeli bile çuvallayacaktır, gerçek hayatta gürültüden tamamen kurtulmak mümkün olmasa da en önemli işlerden biri veriyi mümkün olduğunca gürültüden arındırmaktır.

### Oldukça Eğlenceli Yeni Bir Örnek

Şimdi bu öğrendiklerimizi çok daha ilginç bir örnekte kullanalım, mesela veri setini bir sinüs fonksiyonu kullanarak yaratsak ne olur?

Hemen asıl işe koyulalım, yine aynı şekilde belli bir aralıkta rastgele x değerleri üreteceğiz sonra sin(x) fonksiyonunu kullanarak y değerlerimizi üreteceğiz.

In [ ]:
# Yeni veri setinin üretimi

x_range = (-5, 5) # x değerlerinin alabileceği aralık
n = 300 # 100 adet ölçüm

x = np.linspace(x_range[0], x_range[1], n) # x değerlerini üretelim
y = np.sin(x) # y değerlerini üretelim

plt.scatter(x, y, s=1) # x ve y değerlerini nokta nokta çizdirelim
plt.title("Veri Setimizi Oluşturan Noktalar")
plt.show()

Bu sefer işin güzelliğini bozmamak için gürültü eklemiyoruz ama yine de biraz daha ilginç bir veri seti oluşturmuş olduk.

Asıl sebebi bazılarınız fark etmiş olacak ama fark etmeyenler için asıl sürprizi sona saklayalım, şimdi elimizde lineer ile uzaktan yakından alakası olmayan bir fonksiyon var, $y = m \times x + b$ gibi basit bir formda hiç değil.

Peki sizce bu sefer $\textbf{x}$ vektörünü nasıl oluşturmak mantıklı? Az önce yaptığımız gibi yanına sadece 1 ekleyerek mi? Önce bunu deneyelim.

In [ ]:
# Bu kısımda başta yarattığımız x ve y değerlerini istediğimiz formata dönüştürelim.

# Hatırlayın sadece tek bir ölçümümüz var, noktamızın x eksenindeki konumu, buna göre y'yi bulmak istiyoruz, bunun için x'e bağlı olmayan ikinci bir parametre öğrenmek istiyoruz.
x = x.reshape(-1, 1) # Öncelikle x değerlerini N x 1 boyutlu bir matrise dönüştürüyoruz
ones = np.ones((n, 1)) # N x 1 boyutlu 1'lerden oluşan bir matris oluşturuyoruz

# Şimdi eğer bu iki matrisi yan yana birleştirirsek N x 2 boyutlu bir matris elde ederiz
# Bu matrisin her bir satırında x değerimiz ve 1 değeri olacak yani önceden tartıştığımız formüldeki x ve 1 değerleri
X = np.concatenate((x, ones), axis=1)

# Şimdi de y değerlerini N x 1 boyutlu bir matrise dönüştürüyoruz
Y = y.reshape(-1, 1)

In [ ]:
print("4. Test")
print("Optimal w'yu bulabiliyor muyuz?")

w = find_best_w(X, Y) # En iyi w değerini bulalım
err = E(w, X, Y) # Bulduğumuz w değeri ile hatayı hesaplayalım

print()
print("Bulduğumuz w değeri:")
print(w)

print()
print("Öğrendiğimiz fonksiyonun grafiği:")

x_plot = np.linspace(-5, 5, 1000) # -10 ile 10 arasında 1000 tane x değeri oluşturalım
y_plot_pred = w[0] * x_plot + w[1] # Bu x değerlerine karşılık gelen y değerlerini hesaplayalım
y_plot_real = np.sin(x_plot) # Gerçek y değerlerini hesaplayalım

plt.scatter(x, y, label="Veri", s=3) # Verileri çizelim
plt.scatter(x_plot, y_plot_pred, label="Öğrenilen fonksiyon", s=0.5) # Öğrenilen fonksiyonun grafiğini çizelim
plt.scatter(x_plot, y_plot_real, label="Gerçek fonksiyon", s=0.5) # Gerçek fonksiyonun grafiğini çizelim
plt.legend() # Grafiğin sağ üstünde açıklamaları gösterelim
plt.show() # Grafiği gösterelim




Gördüğünüz gibi bu sefer hiç de iyi bir sonuç elde edemedik, hatta veriyi oluşturan fonksiyonu hiç yakalayamadık bile. Peki neden? Çünkü $\textbf{x}$ vektörümüz artık $y = m \times x + b$ formunda bir fonksiyonu ifade etmiyor, peki önceden bahsettiğimiz polinom örneğindeki gibi bir $P$ sayısı belirleyip $P$ dereceden bir polinom ile öğrenmeyi denesek? Hemen deneyelim.

In [ ]:
# Bu kısımda başta yarattığımız x ve y değerlerini istediğimiz formata dönüştürelim.

# Hatırlayın sadece tek bir ölçümümüz var, noktamızın x eksenindeki konumu, buna göre y'yi bulmak istiyoruz
# Bu sefer x'in 0'dan P'ye kadar kuvvetlerini vektörümüze eklemek istiyoruz ki polinom katsayılarını öğrenebilelim

x = x.reshape(-1, 1) # Öncelikle x değerlerini N x 1 boyutlu bir matrise dönüştürüyoruz

# Şimdi x'in kuvvetlerinden oluşan bir liste oluşturalım, bunun için önce polinom dereceği P'yi belirleyelim
P = 5

powers = [ x ** i for i in range(0, P + 1) ] # x'in 0'dan P'ye kadar kuvvetlerini alalım

# Şimdi de hepsini birleştirip N x (P + 1) boyutlu bir matris elde edelim
X = np.concatenate(powers, axis=1)

# Şimdi de y değerlerini N x 1 boyutlu bir matrise dönüştürüyoruz
Y = y.reshape(-1, 1)

In [ ]:
print("5. Test")
print("Optimal w'yu bulabiliyor muyuz?")

w = find_best_w(X, Y) # En iyi w değerini bulalım
err = E(w, X, Y) # Bulduğumuz w değeri ile hatayı hesaplayalım

print()
print("Bulduğumuz w değeri:")
print(w)

print()
print("Olması gereken w değeri:")
print("""[[-1.85467960e-14]
 [ 8.64102413e-01]
 [ 6.53036653e-15]
 [-1.13925341e-01]
 [-2.87991202e-16]
 [ 2.93910939e-03]]""")

print()
print("Öğrendiğimiz fonksiyonun grafiği:")

x_plot = np.linspace(-5, 5, 1000) # -10 ile 10 arasında 1000 tane x değeri oluşturalım
# Şimdi de bu x değerlerinin kuvvetlerinden oluşan bir matris oluşturalım (Öğrendiğimzi fonksiyon girdi alarak bunu almak zorunda)
x_plot_powers = np.array([ x_plot.reshape(1000, 1) ** i for i in range(0, P + 1) ])
x_plot_matrix = np.concatenate(x_plot_powers, axis=1) # Bu matrisi N x (P + 1) boyutlu bir matrise dönüştürelim

y_plot_pred = x_plot_matrix @ w # Bu x değerlerine karşılık gelen y değerlerini hesaplayalım (@ burada matris çarpımı demek, unutmayın bu matris çarpımı zaten bir tahmin ettiğimiz y'leri veriyor)
# Bu sefer matris çarpımı ile hesapladığımız için y_plot_pred N x 1 boyutlu bir matris olacak, plot etmek için bunu düz bir vektöre çevirelim
y_plot_pred = y_plot_pred.reshape(-1)

y_plot_real = np.sin(x_plot) # Gerçek y değerlerini hesaplayalım

plt.scatter(x, y, label="Veri", s=5) # Verileri çizelim
plt.scatter(x_plot, y_plot_pred, label="Öğrenilen fonksiyon", s=0.5) # Öğrenilen fonksiyonun grafiğini çizelim
plt.scatter(x_plot, y_plot_real, label="Gerçek fonksiyon", s=0.5) # Gerçek fonksiyonun grafiğini çizelim
plt.legend() # Grafiğin sağ üstünde açıklamaları gösterelim
plt.show() # Grafiği gösterelim


### Etkileyici Son

Bu sefer öğrendiğimiz bu fonksiyon gerçeğe oldukça benziyor, peki neden? Taylor açılımını hatırlayalım:

$sin(x) = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + ...$

Yani aslında sin(x) fonksiyonu bir polinom ile ifade edilebilir, hatta bu polinomun derecesini arttırdıkça sin(x) fonksiyonuna daha da yaklaşabiliriz. İşte bu yüzden $\textbf{x}$ vektörümüzü $P$ dereceden bir polinom ile oluşturduğumuzda sin(x) fonksiyonunu oldukça iyi yakaladık.

Hatta ve hatta :)

Sırası ile $x^0$ yani $1$ e karşılık gelen katsayı $-1.85467960 \times 10^{-14} \sim 0$

$x^1$ yani $x$ e karşılık gelen katsayı $8.64102413 \times 10^{-01} \sim 0.864$

$x^2$ ye karşılık gelen katsayı $6.53036653 \times 10^{-15} \sim 0$

$x^3$ ye karşılık gelen katsayı $-1.13925341 \times 10^{-01} \sim -0.113$

$x^4$ ye karşılık gelen katsayı $-2.87991202 \times 10^{-16} \sim 0$

$x^5$ ye karşılık gelen katsayı $2.93910939 \times 10^{-03} \sim 0.00293$

şeklinde.

Peki Taylor açılımında bunlara denk gelen katsayılar neydi?

$ 0, 1, 0, -\frac{1}{3!}, 0, \frac{1}{5!}$

Yani:

$ 0, 1, 0, -0.1666, 0, 0.0083$

Elbette veri sayısını arttırdıkça bu katsayılar daha da yaklaşacaktır (Deneyebilirsiniz).
Veya daha fazla polinom derecesi kullanarak da daha iyi sonuçlar elde edebiliriz. (Deneyebilirsiniz)

#### Kendinize iyi bakın, görüşmek üzere :)